In [1]:
#
#   https://medium.com/@DataPlayer/scalable-approximate-nearest-neighbour-search-using-googles-scann-and-facebook-s-faiss-3e84df25ba
#
import numpy as np
import tensorflow as tf
import scann
import os

# Generate a synthetic dataset of 1 million vectors
NUM_VECTORS = 1000000
VECTOR_DIM  = 128

dataset = np.random.randn(NUM_VECTORS, VECTOR_DIM).astype(np.float32)

2023-10-17 20:42:38.777782: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-17 20:42:38.811080: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-17 20:42:38.812051: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-17 20:42:39.419409: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
#
# Build a ScaNN index with L2 distance metric and 10 random projection hash tables
#
# scann_ops_pybind.builder is a method in the ScaNN library that is used to create a ScannBuilder object.
#
# The builder() method returns a ScannBuilder object,
# which can be used to specify additional parameters such as
#  - the number of projection hash tables to use and
#  - the number of leaves in the hierarchical tree used to organize the data. 

# By default, ScaNN builds a k-means tree, where each leaf node represents a
# cluster of vectors.
# The .tree() method can be used to further split or merge these clusters to
# optimize the search performance.

searcher = scann.scann_ops_pybind.builder(
    dataset,                             # numpy array containing the data points to be indexed
    num_neighbors    =  10,              # the number of neighbors to search for
    distance_measure = "squared_l2"      # the distance metric to use (e.g., 'dot_product' or 'squared_l2')
).tree(                                  # .tree() is a method that can be used after(?) calling .build() to further refine the tree structure of the index. .tree() allows us to fine-tune the tree structure of the index to optimize the search performance for a given dataset and use case. It takes several arguments: 
    num_leaves           =   2000,       # num_leaves: the maximum number of leaves in the tree . This can be used to control the balance between search speed and memory usage. Increasing num_leaves will improve the search accuracy but also increase the memory usage.
    num_leaves_to_search =    100,       #num_leaves_to_search: the number of leaves to search during the query phase
    training_sample_size = 250000        # training_sample_size: the number of vectors to use for training the tree.
).score_ah(                              # score_ah() is a method in ScaNN that sets the parameters for the asymmetric hash function used in the indexing process. It is called after .tree() because the tree structure is required to set these parameters.
    2,                                   # num_neighbors sets the number of neighbors to consider when selecting the threshold for asymmetric hashing   
    anisotropic_quantization_threshold = 0.2 # anisotropic_quantization_threshold controls how many different quantization levels to use for each dimension in the hash function (maller values generally resulting in better accuracy at the cost of higher indexing time and memory usage.)
).reorder(
    100
).build()  # Finally, the ScannBuilder object's build() method is called to create the index.

2023-10-17 22:07:58.988362: I scann/partitioning/partitioner_factory_base.cc:59] Size of sampled dataset for training partition: 249544
2023-10-17 22:08:06.575689: I ./scann/partitioning/kmeans_tree_partitioner_utils.h:84] PartitionerFactory ran in 7.587267959s.


In [3]:
# Create a directory for the serialized index

if not os.path.exists("scann_index"):
    os.makedirs("scann_index")


In [4]:
# Save the index to disk

searcher.serialize("scann_index")

In [ ]:
# Load the index from disk
#searcher = scann.scann_ops_pybind.load_searcher("scann_index/")

In [6]:
# Generate a query vector
query_vector = np.random.randn(VECTOR_DIM).astype(np.float32)

In [8]:
# Find the top 10 nearest neighbors of the query vector
neighbors, distances = searcher.search(query_vector, final_num_neighbors=10)

In [9]:
# neighbours is an array of indexes for nearest neighbors. distances is an array with their corresponding squared_L2 distnaces
neighbors,distances

(array([732276, 981817, 598507, 510868, 730527,   2208, 122018, 973694,
        741261, 231291], dtype=uint32),
 array([150.53412, 154.43344, 156.3534 , 156.4636 , 158.85886, 163.04324,
        163.57863, 163.59833, 163.9052 , 164.5318 ], dtype=float32))